In [1]:
import fast_rl.core.Interpreter
import fast_rl.core.Learner 
import fast_rl.agents.DQN 
from fast_rl.agents.DQN import DQN, FixedTargetDQN, DoubleDQN, DuelingDQN, DoubleDuelingDQN
from fast_rl.core.Interpreter import AgentInterpretationAlpha
from fast_rl.core.Learner import AgentLearner
from fast_rl.core.MarkovDecisionProcess import MDPDataBunch
from fast_rl.core.agent_core import PriorityExperienceReplay, ExperienceReplay
from fast_rl.core.MarkovDecisionProcess import MDPDataBunch, FEED_TYPE_IMAGE, FEED_TYPE_STATE
from fast_rl.core.agent_core import ExperienceReplay, GreedyEpsilon
import sys
import importlib

In [2]:
data = MDPDataBunch.from_env('maze-random-5x5-v0', render='human', max_steps=1000)
model = FixedTargetDQN(data, batch_size=128, max_episodes=20, lr=0.001, copy_over_frequency=3,
                       memory=ExperienceReplay(10000), discount=0.99, 
                      exploration_strategy=GreedyEpsilon(epsilon_start=1, epsilon_end=0.1,
                                                         decay=0.0001, do_exploration=True))
learn = AgentLearner(data, model)

learn.fit(4)

pygame 2.0.0.dev3 (SDL 2.0.9, python 3.6.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


epoch,train_loss,valid_loss,epsilon,time
0,0.000004,0.000004,0.948945,00:51
1,0.000005,0.000005,0.936473,00:24
2,0.000001,0.000001,0.919582,00:17
3,0.000001,0.000001,0.897989,00:29


In [3]:
interp = AgentInterpretationAlpha(learn, base_chart_size=(20, 10))
interp.plot_heatmapped_episode(-1, return_heat_maps=False)

AttributeError: 'MarkovDecisionProcessList' object has no attribute 'itmes'

In [ ]:
for i in range(4):
    interp.plot_heatmapped_episode(-1, action_index=i, return_heat_maps=False)

In [ ]:
interp.plot_q_density()